In [7]:
import os, pandas as pd, stanza, pathlib
from tqdm import tqdm
from stanza.utils.conll import CoNLL

In [ ]:
BASE_DIR = pathlib.Path('C:/Users/Vignir/PycharmProjects/SUDcom/dataset')  
FILES = [
    BASE_DIR / 'A2.csv',
    BASE_DIR / 'B2.csv',
    BASE_DIR / 'C2.csv',
    BASE_DIR / 'cefr_leveled_texts.csv',    
]
LANG  = "en"

In [3]:
dfs = []
for f in FILES:
    if os.path.exists(f):
        dfs.append(pd.read_csv(f, encoding="utf-8"))
if not dfs:
    raise FileNotFoundError("None of the CSVs were found. Check FILES paths.")
data = pd.concat(dfs, ignore_index=True)
if "text" not in data.columns:
    raise ValueError("CSV must contain a 'text' column.")
data = data.dropna(subset=["text"]).reset_index(drop=True)


In [4]:
try:
    stanza.download(LANG, processors="tokenize,mwt,pos,lemma,depparse", verbose=False)
except Exception:
    pass
nlp = stanza.Pipeline(lang=LANG, processors="tokenize,pos,lemma,depparse", use_gpu=True, verbose=False)

In [10]:
%env OMP_NUM_THREADS=4
rows = []
data = data.head(200)
for i, txt in tqdm(enumerate(data["text"].astype(str), start=1), total=len(data)):
    doc = nlp(txt)
    lab = data["label"][i-1] if "label" in data.columns else None
    for j, sent in enumerate(doc.sentences, start=1):
        for w in sent.words:
            rows.append({
                "doc_id": i,
                "sent_id": j,
                "token_id": w.id,
                "form": w.text,
                "lemma": w.lemma,
                "upos": w.upos,
                "xpos": w.xpos or "_",
                "head": w.head,
                "deprel": w.deprel,
                **({"label": lab} if lab is not None else {})
            })

tagged = pd.DataFrame(rows)
display(tagged.head(20))

env: OMP_NUM_THREADS=4


 33%|███▎      | 66/200 [00:50<01:42,  1.31it/s]


KeyboardInterrupt: 

In [8]:
conll = CoNLL.convert_dict(tagged.to_dict)

TypeError: 'method' object is not iterable